In [ ]:
!pip install -q -U transformers datasets bitsandbytes peft

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

In [2]:
import huggingface_hub

huggingface_hub.login(token=HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset
dataset = load_dataset("BLACKBUN/old_korean_newspaper_1897_1910_economy_politic_qa")

Generating train split:   0%|          | 0/8946 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/284 [00:00<?, ? examples/s]

In [ ]:
# for 2 billion model

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

BASE_MODEL = "google/gemma-2-2b-it"
ADAPTER = "jia6776/korean_history_1897_1910_gemma2_2b_lora"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

model = PeftModel.from_pretrained(model, ADAPTER).to(device)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

BASE_MODEL = "google/gemma-2-9b-it"
ADAPTER = "BLACKBUN/korean_history_1897_1910_gemma2_9b_lora_q4"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto"  # This will automatically distribute the model across available GPUs
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# Load the LoRA adapter
model = PeftModel.from_pretrained(model, ADAPTER)

In [ ]:
question = dataset['test']['Question'][10]
answer = dataset['test']['Answer'][10]

In [ ]:
print(f"Q:{question}")

In [ ]:
messages = [
    # {
    #     "role": "system",
    #     "content": "You are a helpful assistant."
    # },
    {
        "role": "user",
        "content": f"{question}"
    },

]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
input_text = prompt
inputs = tokenizer(input_text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_length=256)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(f"Model Answer: {generated_text}\n")
print(f"Ground Answer: {answer}")

In [ ]:
pipe_finetuned = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, device="cuda")